<a href="https://colab.research.google.com/github/thisishasan/bigdata_mod_b/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [250]:
#Download the dataset
!wget -nc 'https://raw.githubusercontent.com/thisishasan/bigdata_mod_b/refs/heads/main/dataset_aquifers_fe.csv'

File ‘dataset_aquifers_fe.csv’ already there; not retrieving.



In [251]:
#Load the dataset into a dataframe
import pandas as pd
df = pd.read_csv("dataset_aquifers_fe.csv")

In [252]:
df.columns = df.columns.str.lower()

In [253]:
#Dataset structure
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8154 entries, 0 to 8153
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   mean_rainfall      8154 non-null   float64
 1   mean_temp          8154 non-null   float64
 2   actual_depth       8154 non-null   float64
 3   actual_volume      8154 non-null   float64
 4   actual_hydrometry  8154 non-null   float64
 5   date               8154 non-null   object 
dtypes: float64(5), object(1)
memory usage: 382.3+ KB


In [254]:
#Convert data type of 'Date' column into datetime format
df['date'] = pd.to_datetime(df['date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8154 entries, 0 to 8153
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   mean_rainfall      8154 non-null   float64       
 1   mean_temp          8154 non-null   float64       
 2   actual_depth       8154 non-null   float64       
 3   actual_volume      8154 non-null   float64       
 4   actual_hydrometry  8154 non-null   float64       
 5   date               8154 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(5)
memory usage: 382.3 KB


In [255]:
#Print top five (05) rows of the dataset
df.head()

,mean_rainfall,mean_temp,actual_depth,actual_volume,actual_hydrometry,date
0,0.415556,6.6250,-6.08026,-8019.271158,-0.083056,1998-01-04
1,2.054444,6.0750,-6.06452,-7956.571285,-0.104167,1998-01-05
2,0.921111,9.0875,-6.15706,-7715.808854,0.011944,1998-01-06
3,0.878889,12.3250,-6.10774,-7731.378766,-0.008611,1998-01-07
4,0.908889,12.6500,-6.05310,-7812.676449,-0.072222,1998-01-08


In [256]:
#Check empty values in the dataframe
df.isnull().sum()

,0
mean_rainfall,0
mean_temp,0
actual_depth,0
actual_volume,0
actual_hydrometry,0
date,0


In [257]:
#Check duplicate values in the dataframe
df.duplicated().sum()

np.int64(0)

In [258]:
#Select sample of records (between full calendar years 2015 to 2019)
sample_df = df[(df['date'] >= '2015-01-01') & (df['date'] <= '2019-12-31')]
sample_df.head()

,mean_rainfall,mean_temp,actual_depth,actual_volume,actual_hydrometry,date
6146,1.316667,3.955694,-32.820496,-7698.896136,1.5750,2015-01-01
6147,5.357520,5.879062,-31.388567,-7925.015531,1.4200,2015-01-02
6148,0.470556,10.397986,-31.843199,-8400.301708,1.3600,2015-01-03
6149,0.102778,11.502569,-32.979759,-8941.250075,1.2875,2015-01-04
6150,0.105556,12.872986,-32.836579,-7591.328136,1.4050,2015-01-05


In [259]:
sample_df.tail()

,mean_rainfall,mean_temp,actual_depth,actual_volume,actual_hydrometry,date
7967,0.070000,5.999375,-33.881034,-7469.499812,1.1300,2019-12-27
7968,0.075000,5.216250,-33.901214,-6378.523936,1.1025,2019-12-28
7969,0.035000,3.274375,-33.751720,-7615.947150,0.9925,2019-12-29
7970,0.125000,3.742500,-33.846299,-7231.706197,1.2775,2019-12-30
7971,0.040833,5.622083,-33.883129,-7305.354072,1.2150,2019-12-31


In [260]:
# Check for missing dates
sample_df.set_index('date', inplace=True)
full_range = pd.date_range(start='2015-01-01', end='2019-12-31', freq='D')
sample_df = sample_df.reindex(full_range)
missing_dates = sample_df[sample_df.isnull().any(axis=1)]
print("Number of missing dates:", len(missing_dates))
print("Missing dates:\n", missing_dates.index)
sample_df.head()

Number of missing dates: 0
Missing dates:
 DatetimeIndex([], dtype='datetime64[ns]', freq='D')


,mean_rainfall,mean_temp,actual_depth,actual_volume,actual_hydrometry
2015-01-01,1.316667,3.955694,-32.820496,-7698.896136,1.5750
2015-01-02,5.357520,5.879062,-31.388567,-7925.015531,1.4200
2015-01-03,0.470556,10.397986,-31.843199,-8400.301708,1.3600
2015-01-04,0.102778,11.502569,-32.979759,-8941.250075,1.2875
2015-01-05,0.105556,12.872986,-32.836579,-7591.328136,1.4050


In [261]:
import statsmodels.api as sm